In [1]:
from configparser import ConfigParser
from mysql.connector import MySQLConnection,Error

**Config reading and parsing**

In [2]:
# reading database credentials from config.ini file

def read(filename='config.ini',section='mysql'):
    parser=ConfigParser()
    parser.read(filename)
    
    db={}
    
    if parser.has_section(section):
        items=parser.items(section)
        for item in items:
            db[item[0]]=item[1]
    else:
        raise Exception(f'{section} not found in file {filename}')
    return db 


print(read(filename="config.ini",section="mysql"))

{'host': '127.0.0.1', 'port': '99966', 'database': 'books', 'user': 'root', 'password': 'Milla123!'}


**Connecting with MySQL/MariaDB database server and getting the connection and cursor object**

In [3]:
def connect(creds):
    con=None
    try:
        print('Connecting to MySQL database...')
        con=MySQLConnection(**creds)
        
        
        if con.is_connected():
            print('Connection established')
            cus = con.cursor(buffered=True)
        else:
            print('Connection failed')
            
    except Error as e:
        print(e)
    finally:
        return con,cus
    
cn,cs=connect(creds=read(filename="config.ini",section="mysql"))

Connecting to MySQL database...
Connection established


**Genres List**

In [4]:
genres_lookup_list = ['biography',
                      'children',
                      'comics',
                      'crime',
                      'fantasy',
                      'fiction',
                      'graphic',
                      'historical fiction',
                      'history',
                      'mystery',
                      'non-fiction',
                      'paranormal',
                      'poetry',
                      'romance',
                      'thriller',
                      'young-adult']

In [5]:
def sp_create_genres_lookup(data,cs,proc_name="sp_create_genres_lookup",initial=[0,1],end=[0]):
    
    procs_args_complete = []

    procs_args_complete.extend(initial)
    procs_args_complete.extend(data)
    procs_args_complete.extend(end)

    upload_details = cs.callproc(procname=proc_name, args=procs_args_complete)

    return upload_details

In [6]:
records = []

for i in genres_lookup_list:
    name = i
    desc = i
    records.append((name,desc)) 

In [7]:
records

[('biography', 'biography'),
 ('children', 'children'),
 ('comics', 'comics'),
 ('crime', 'crime'),
 ('fantasy', 'fantasy'),
 ('fiction', 'fiction'),
 ('graphic', 'graphic'),
 ('historical fiction', 'historical fiction'),
 ('history', 'history'),
 ('mystery', 'mystery'),
 ('non-fiction', 'non-fiction'),
 ('paranormal', 'paranormal'),
 ('poetry', 'poetry'),
 ('romance', 'romance'),
 ('thriller', 'thriller'),
 ('young-adult', 'young-adult')]

**Uploading the Complete Data into the Database**

In [8]:
for i in records:
    record = i
    sp_create_genres_lookup(data=record,cs=cs,proc_name="sp_create_genres_lookup",initial=[0,1],end=[0])

cn.commit()